In [ ]:
pip install transformers

In [ ]:
!pip install gdown

학습 데이터와 테스트 데이터 다운로드 방법은 e-book을 통해서만 제공합니다.

In [ ]:
import pandas as pd

In [ ]:
DATA_TRAIN_PATH = 'summ_train.json'
train_df = pd.read_json(DATA_TRAIN_PATH)
train_df = train_df.dropna()
train_df = train_df[:40000]
len(train_df)

40000

In [ ]:
DATA_TEST_PATH = 'summ_test.json'
test_df = pd.read_json(DATA_TEST_PATH)
test_df = test_df.dropna()
test_df = test_df[:5000]
len(test_df)

5000

In [ ]:
def preprocess_data(data):
    outs = []
    for doc in data['documents']:
        line = []
        line.append(doc['media_name'])
        line.append(doc['id'])
        para = []
        for sent in doc['text']:
            for s in sent:
                para.append(s['sentence'])
        line.append(para)
        line.append(doc['abstractive'][0])
        line.append(doc['extractive'])
        a = doc['extractive']
        if a[0] == None or a[1] == None or a[2] == None:
            continue
        outs.append(line)

    outs_df = pd.DataFrame(outs)
    outs_df.columns = ['media', 'id', 'article_original', 'abstractive', 'extractive']
    return outs_df

In [ ]:
# 원문과 요약문을 각각 'article_original'와 'abstractive'열에 저장.
train_data = preprocess_data(train_df)
train_data.head()

,media,id,article_original,abstractive,extractive
0,광양신문,290741778,"[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...",전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...,"[2, 3, 10]"
1,광양신문,290741792,"[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...,"[2, 4, 11]"
2,광양신문,290741793,"[전남드래곤즈 해맞이 다짐…선수 영입 활발, 이성훈 sinawi@hanmail.ne...",전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...,"[3, 5, 7]"
3,광양신문,290741794,"[11~24일, 매실·감·참다래 등 지역특화작목, 이성훈 sinawi@hanmail...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작...","[2, 3, 4]"
4,광양신문,290741797,"[홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항, 이성훈 sinawi@han...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...,"[3, 7, 4]"


In [ ]:
# 원문과 요약문을 각각 'article_original'와 'abstractive'열에 저장.
test_data = preprocess_data(test_df)
test_data.head()

,media,id,article_original,abstractive,extractive
0,한국경제,340626877,"[[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, ...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...,"[0, 1, 3]"
1,한국경제,340626896,"[] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다., 바른미래당이 8일 ...",8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...,"[2, 1, 6]"
2,한국경제,340626904,"[[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀., 지난 3일 한국이 세계 최초...",지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...,"[1, 5, 8]"
3,한국경제,340627450,[] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...,"[0, 1, 2]"
4,한국경제,340627465,"[[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를...",주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...,"[1, 3, 4]"


In [ ]:
# train_data의 첫번째 샘플의 article_original 열의 값 출력
train_data['article_original'].loc[0]

['ha당 조사료 400만원…작물별 차등 지원',
 '이성훈 sinawi@hanmail.net',
 '전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다.',
 '쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다.',
 '올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다.',
 '지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다.',
 '지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다.',
 '농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다.',
 'ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다.',
 '벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다.',
 '논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다.',
 '전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다.',
 '최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.']

In [ ]:
train_data['news'] = train_data['article_original'].apply(lambda x: ' '.join(x))
test_data['news'] = test_data['article_original'].apply(lambda x: ' '.join(x))

In [ ]:
# train_data의 첫번째 샘플의 news 열의 값 출력
train_data['news'].loc[0]

'ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.'

In [ ]:
train_data[['news', 'abstractive']].head()

,news,abstractive
0,ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...
1,"8억 투입, 고소천사벽화·자산마을에 색채 입혀 이성훈 sinawi@hanmail.n...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...
2,전남드래곤즈 해맞이 다짐…선수 영입 활발 이성훈 sinawi@hanmail.net ...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...
3,"11~24일, 매실·감·참다래 등 지역특화작목 이성훈 sinawi@hanmail.n...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작..."
4,"홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항 이성훈 sinawi@hanma...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...


In [ ]:
import logging
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from transformers import T5ForConditionalGeneration, T5TokenizerFast
from tqdm import tqdm

In [ ]:
class T5SummaryDataset(Dataset):
    def __init__(self, df, tokenizer, max_len, ignore_index=-100):
        super().__init__()
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.docs = df
        self.len = self.docs.shape[0]
        self.pad_index = self.tokenizer.pad_token_id
        self.ignore_index = ignore_index

    def add_padding_data(self, inputs):
        if len(inputs) < self.max_len:
            pad = np.array([self.pad_index] * (self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:self.max_len]
        return inputs

    def add_ignored_data(self, inputs):
        if len(inputs) < self.max_len:
            pad = np.array([self.ignore_index] * (self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:self.max_len]
        return inputs

    def __getitem__(self, idx):
        instance = self.docs.iloc[idx]
        input_text = "summarize: " + instance['news']
        input_ids = self.tokenizer.encode(input_text, return_tensors="pt", max_length=self.max_len, truncation=True).squeeze()
        input_ids = self.add_padding_data(input_ids)

        label_ids = self.tokenizer.encode(instance['abstractive'], return_tensors="pt", max_length=self.max_len, truncation=True).squeeze()
        label_ids = self.add_ignored_data(label_ids)

        return {'input_ids': np.array(input_ids, dtype=np.int_),
                'labels': np.array(label_ids, dtype=np.int_)}

    def __len__(self):
        return self.len

In [ ]:
class T5ConditionalGeneration(torch.nn.Module):
    def __init__(self):
        super(T5ConditionalGeneration, self).__init__()
        self.model = T5ForConditionalGeneration.from_pretrained('paust/pko-t5-base')
        self.tokenizer = T5TokenizerFast.from_pretrained('paust/pko-t5-base')
        self.pad_token_id = self.tokenizer.pad_token_id

    def forward(self, inputs):
        attention_mask = inputs['input_ids'].ne(self.pad_token_id).float()
        return self.model(input_ids=inputs['input_ids'],
                          attention_mask=attention_mask,
                          labels=inputs['labels'], return_dict=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = T5ConditionalGeneration().to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
batch_size = 8
max_len = 512
num_workers = 4
lr = 3e-5
max_epochs = 10
warmup_ratio = 0.1

tokenizer = T5TokenizerFast.from_pretrained('paust/pko-t5-base')
train_dataset = T5SummaryDataset(train_data, tokenizer, max_len=max_len)
test_dataset = T5SummaryDataset(test_data, tokenizer, max_len=max_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers)

optimizer = AdamW(model.parameters(), lr=lr)
total_steps = len(train_loader) * max_epochs
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=int(total_steps * warmup_ratio), T_mult=1, eta_min=0)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
print('T5의 패딩 토큰 번호 :', tokenizer.pad_token_id)

T5의 패딩 토큰 번호 : 0


In [ ]:
print('첫번째 샘플의 원본 텍스트 :', train_data['news'].loc[0])

첫번째 샘플의 원본 텍스트 : ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.


In [ ]:
print('첫번째 샘플의 정수 인코딩과 패딩 후의 결과 :', train_dataset[0]['input_ids'])

첫번째 샘플의 정수 인코딩과 패딩 후의 결과 : [ 7675    78 20359    74 26159    27   222  7231   480   222  1526   541
   222  7004    17 23431    15    15    15 13238   681   222 18165   222
   926   222  5009  1303   222    84  2667    66  7066    33    73  3415
 11261 11866    15 39816   222  5809  6171   278   222  2533   222 12050
   871   333   222  6778   403   373   222  1745   701   222   863   222
  1387   667   222  2642   429   222  2533   222  1951  2487   354   333
   222  2430   222  1808  4210   222   500    15   222  2533   222  1951
  2487   354   274   222  1650   333   222   527  1963   222  1283   279
   222  1650   222  2233   222  4758 13238   824   222  1967   222   450
   222   804   222 13238   291   222 49511   222  1650  2911   222  1928
   222  2470   466   333   222   336  3502  1116   222  2452   267    15
   222  1387   222  4147   302   222  1283   222   804   222 13238   222
  5431   222  1247  4107   311   222  1404   222    22   348  7231   302
   222   585   222  3738

In [ ]:
print('첫번째 샘플의 정수 인코딩 결과를 텍스트로 복원 후 :')
tokenizer.decode(train_dataset[0]['input_ids'])

첫번째 샘플의 정수 인코딩 결과를 텍스트로 복원 후 :


'summarize: ha당 조사료 400만원...작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논</s>'

In [ ]:
print('첫번째 샘플의 원본 요약문:')
train_data['abstractive'].loc[0]

첫번째 샘플의 원본 요약문:


"전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다 ."

In [ ]:
print('첫번째 샘플의 요약문의 정수 인코딩 및 -100으로 패딩한 결과')
train_dataset[0]['labels']

첫번째 샘플의 요약문의 정수 인코딩 및 -100으로 패딩한 결과


array([ 5809,  6171,   278,   222,  2533,   222, 12050,   871,   333,
         222,  6778,   403,   373,   222,  1745,   701,   222,   863,
         222,  1387,   667,   222,  1650,   333,   222,   527,  1963,
         222,  1283,   279,   222,  1650,   222,  2233,   222,  4758,
       13238,   824,   222,  1967,   222,   450,   222,   804,   222,
       13238,   291,   222, 49511,   222,  1650,  2911,   222,  1928,
         222,  2470,   466,   333,   222,   336,  3502,  1116,   222,
           8,  2533,   222,  1951,  2487,   354,     8,   333,   222,
        2430,   403,   373,   222,  2642,   701,   293,   222,   443,
         222,  1243,   222,  3858,   349,   667,   222,    19,   515,
         222,  3732,   349,   579,   222, 14205,   222,  2500,   284,
         222, 30492,  1588,   402,   389,   222,  1143,  6604,   222,
          15,     1,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
        -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
        -100,  -100,

In [ ]:
# -100의 값은 tokenizer_decode를 하면 에러나므로 임시로 0으로 변경 후 출력
test_array = train_dataset[0]['labels']
test_array[test_array == -100] = 0
print('첫번째 샘플의 요약문의 정수 인코딩 결과를 텍스트로 복원 후 :')
tokenizer.decode(test_array)

첫번째 샘플의 요약문의 정수 인코딩 결과를 텍스트로 복원 후 :


"전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [ ]:
best_loss = np.inf
for epoch in range(max_epochs):
    print(epoch+1, '수행 중')
    model.train()
    for batch in tqdm(train_loader, total=len(train_loader)):
        batch = {k: v.to(device) for k, v in batch.items()} # Move the batch tensors to the same device as the model
        optimizer.zero_grad()
        outputs = model(batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(test_loader, total=len(test_loader)):
            batch = {k: v.to(device) for k, v in batch.items()} # Move the batch tensors to the same device as the model
            outputs = model(batch)
            total_loss += outputs.loss.item()

    avg_loss = total_loss / len(test_loader)
    print(f'Epoch: {epoch+1}, Loss: {avg_loss}')

    # Save the best model
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), 'best_model.pt')
        print(f'Validation loss improved from {best_loss:.4f} to {avg_loss:.4f}. 체크포인트를 저장합니다.')

1 수행 중


100%|██████████| 625/625 [01:50<00:00,  5.67it/s]


Epoch: 1, Loss: 0.9526667984962464
Validation loss improved from 0.9527 to 0.9527. 체크포인트를 저장합니다.
2 수행 중


100%|██████████| 625/625 [01:50<00:00,  5.67it/s]


Epoch: 2, Loss: 0.9351357998371125
Validation loss improved from 0.9351 to 0.9351. 체크포인트를 저장합니다.
3 수행 중


 33%|███▎      | 1655/5000 [15:32<31:24,  1.77it/s]


KeyboardInterrupt: ignored

In [ ]:
# 모델 인스턴스 생성
model_wrapper = T5ConditionalGeneration().to(device)

# 가중치 로드
model_wrapper.load_state_dict(torch.load('best_model.pt'))

# 모델을 평가 모드로 설정
model_wrapper.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ConditionalGeneration(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(50358, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(50358, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedActDense(
                (wi_0): Linear(in_features=768, out_features=2048, bias=False)
                (wi_

In [ ]:
text = test_data.loc[25]['news']
text

"배우 배수지가 매니지먼트 숲과 전속계약을 체결했다. 수지는 8일 자신의 인스타그램에 '데뷔 때 부터 함께해온 소속사 JYP와 계약기간을 마치고 오늘부터 새로운 소속사 매니지먼트 숲과 함께 하게 되었다'고 밝혔다. 이어 수지는 '연습생으로 시작해서, 데뷔하고 9년의 시간이 흐른 지금까지, JYP와 함께했던 여러 영광의 순간들이 스쳐지나간다'면서 '9년 동안 항상 옆에서 서포트 해주셨던 JYP 모든 직원분들께 진심으로 감사드린다'고 인사를 잊지 않았다. 2010년 걸그룹 '미쓰에이'로 데뷔한 배수지는 2011년 KBS2 드라마 '드림하이'로 첫 연기 활동을 시작했다. 2012년 영화 '건축학개론'을 통해 스크린 데뷔를 한 뒤 가수 활동과 연기 활동을 꾸준히 병행해 오고 있다. 매니지먼트 숲 관계자는 '배우 배수지의 장점과 매력을 극대화할 수 있는 작품 선택부터 국내외 활동, 가수로서의 솔로 활동까지 활발하게 이루어질 수 있도록 지원할 예정이다'고 전했다. 특히 올해는 작품을 통해 연기자 배수지로 대중들과 만날 예정이다. 현재 촬영 중인 SBS 드라마 '배가본드'는 민항 여객기 추락 사고에 연루된 한 남자가 은폐된 진실 속에서 찾아낸 거대한 국가 비리를 파헤치게 되는 과정을 담은 이야기다. 배수지는 국정원 블랙요원 고해리 역으로 출연하며, 뒤이어 영화 '백두산'에도 합류한다. 매니지먼트 숲은 공유, 공효진, 김재욱, 서현진, 이천희, 전도연, 정유미, 남지현, 최우식, 유민규, 이재준, 정가람, 전소니 등 소속되어 있다."

In [ ]:
text = "summarize: " + text

In [ ]:
input_ids = tokenizer.encode(text)
tokenizer.decode(input_ids)

"summarize: 배우 배수지가 매니지먼트 숲과 전속계약을 체결했다. 수지는 8일 자신의 인스타그램에 '데뷔 때 부터 함께해온 소속사 JYP와 계약기간을 마치고 오늘부터 새로운 소속사 매니지먼트 숲과 함께 하게 되었다'고 밝혔다. 이어 수지는 '연습생으로 시작해서, 데뷔하고 9년의 시간이 흐른 지금까지, JYP와 함께했던 여러 영광의 순간들이 스쳐지나간다'면서 '9년 동안 항상 옆에서 서포트 해주셨던 JYP 모든 직원분들께 진심으로 감사드린다'고 인사를 잊지 않았다. 2010년 걸그룹 '미쓰에이'로 데뷔한 배수지는 2011년 KBS2 드라마 '드림하이'로 첫 연기 활동을 시작했다. 2012년 영화 '건축학개론'을 통해 스크린 데뷔를 한 뒤 가수 활동과 연기 활동을 꾸준히 병행해 오고 있다. 매니지먼트 숲 관계자는 '배우 배수지의 장점과 매력을 극대화할 수 있는 작품 선택부터 국내외 활동, 가수로서의 솔로 활동까지 활발하게 이루어질 수 있도록 지원할 예정이다'고 전했다. 특히 올해는 작품을 통해 연기자 배수지로 대중들과 만날 예정이다. 현재 촬영 중인 SBS 드라마 '배가본드'는 민항 여객기 추락 사고에 연루된 한 남자가 은폐된 진실 속에서 찾아낸 거대한 국가 비리를 파헤치게 되는 과정을 담은 이야기다. 배수지는 국정원 블랙요원 고해리 역으로 출연하며, 뒤이어 영화 '백두산'에도 합류한다. 매니지먼트 숲은 공유, 공효진, 김재욱, 서현진, 이천희, 전도연, 정유미, 남지현, 최우식, 유민규, 이재준, 정가람, 전소니 등 소속되어 있다.</s>"

In [ ]:
input_ids = tokenizer.encode(text)
input_ids = torch.tensor(input_ids)
input_ids = input_ids.unsqueeze(0).to(device)
output = model_wrapper.model.generate(input_ids, eos_token_id=1, max_length=512, num_beams=5)
output = tokenizer.decode(output[0], skip_special_tokens=True)
print(output)

배우 배수지가 8일 자신의 인스타그램에 '데뷔 때 부터 함께해온 소속사 JYP와 계약기간을 마치고 오늘부터 새로운 소속사 매니지먼트 숲과 함께 하게 되었다'며 '9년 동안 항상 옆에서 서포트 해주셨던 JYP 모든 직원분들께 진심으로 감사드린다'고 인사를 전했다.
